In [ ]:
### here we import various libraries needed
import pandas as pd
import numpy as np
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps

In [ ]:
sql='''SELECT
presc.month,
pct,
pract.name,
presc.practice,
AVG(total_list_size) AS list_size,
presc.bnf_code,
presc.bnf_name,
SUM(items) AS totalitems,
SUM(actual_cost) AS totalcost
FROM
ebmdatalab.hscic.normalised_prescribing_standard AS presc
LEFT JOIN
ebmdatalab.hscic.practices AS pract
ON
presc.practice=pract.code
LEFT JOIN  # is this join in the right place  review
ebmdatalab.hscic.practice_statistics_all_years AS stat
ON
presc.practice = stat.practice
AND presc.month = stat.month
LEFT JOIN
ebmdatalab.dmd.product AS dmd
ON
presc.bnf_code=dmd.bnf_code
WHERE
dmd.is_blacklisted = true
GROUP BY
month,
pct,
pract.name,
presc.practice,
presc.bnf_code,
presc.bnf_name'''


df_blacklist = bq.cached_read(sql, csv_path='df_blacklist.csv', use_cache=True)  # add `use_cache=False` to override
df_blacklist['month'] = df_inhaler['month'].astype('datetime64[ns]')
df_blacklist.head()